In [374]:
#imports
from optparse import Option
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By

#BET365
from time import sleep
import undetected_chromedriver as uc

#from ast import main
import string
from tkinter import END, Y
from selenium import webdriver
#from random_user_agent.user_agent import UserAgent
#from random_user_agent.params import SoftwareName,OperatingSystem
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import copy
import datetime
import os

import random
from collections import namedtuple
import copy
import numpy as np

from requests import options
from selenium import webdriver
import chromedriver_autoinstaller
from undetected_chromedriver import ChromeOptions

from collections import namedtuple
import json


In [375]:
#Methods


#Funzione che genera la media delle nuove bid in base al range dei prezzi di un prodotto su Google Shopping

#Ho considerato i valori delle bids compresi tra 2 e 5 ed ho fatto in modo che il nuovo valore della bid fosse proporzionale al price.
def bid_generator(OldMax,OldMin,NewMax,NewMin,OldValue):
    OldRange = (OldMax - OldMin)  
    NewRange = (NewMax - NewMin)  
    NewValue = (((OldValue - OldMin) * NewRange) / OldRange) + NewMin
    return NewValue

#Funzione max
def max_fun(datas):
    max = datas[0].price
    for i in range(len(datas)):
        if datas[i].price > max:
            max = datas[i].price
    return max


#Funzione min
def min_fun(datas):
    min = datas[0].price
    for i in range(len(datas)):
        if datas[i].price < min:
            min = datas[i].price
    return min

#Funzione per fare scraping e prendere prodotto/competior/prezzi
def scraping_Google(link_product):
    chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                        # and if it doesn't exist, download it automatically,
                                        # then add chromedriver to path
    options = ChromeOptions()
    options.add_argument('--no-sandbox')

    driver = webdriver.Chrome(options=options)
    driver.get(link_product)

    accedi = driver.find_element(By.XPATH,'//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[4]/div[1]/div[1]/form[2]/div/div/button/span')
    accedi.click()

    sleep(2)

    products = driver.find_elements(By.CLASS_NAME,'KZmu8e')
    competitors = driver.find_elements(By.CLASS_NAME,'E5ocAb')
    prices = driver.find_elements(By.CLASS_NAME,'T14wmb')

    return prices


#len, indica il numero di prezzi da fare scraping
def list_prices_bids(prices,len,var):
    ncomp = namedtuple('Competitor', ['id', 'price','bid'])
    datas = []

    #for i in range(len(prices)):
    for i in range(len):

        
        appoggio = prices[i].text
        appoggio = appoggio.replace(" €","")
        appoggio = appoggio.replace(",",".")
        if " Ricondizionato" in appoggio:
            appoggio = appoggio.replace(" Ricondizionato","")
        if " Usato" in appoggio:
            appoggio = appoggio.replace(" Usato","")
        if "Usato" in appoggio:
            appoggio = appoggio.replace("Usato","")

 
        appoggio = fix_sales_price(appoggio)

        datas.append(ncomp(i,appoggio,1))

  

    datas[i] = datas[i]._replace(bid=round(1,2))
    
    return datas


#Questa funzione serve per correggere lo scraping di alcuni prezzi che fanno crashare poichè c'è lo sconto e si stampano cosi 13.3434.23
def fix_sales_price(price):
    price = price.split('.')

    price = price[0]+"."+price[1]

    return round(float(price),2)


#Funzione che mi scrive il dataset su file json
def json_write(dataset,filename):
    dumped = []
    for elem in dataset:
        dumped.append(json.dumps(elem._asdict()))

    print(dumped)

    # Directly from dictionary
    with open(filename, 'w') as outfile:
        json.dump(dumped, outfile)
    


In [411]:
link_tazze = 'https://www.google.it/search?q=tazza+calcio&sa=X&biw=1200&bih=853&tbm=shop&ei=2j6cZIfPKpWJxc8P_cSM2Ag&ved=0ahUKEwiHpNLwkeb_AhWVRPEDHX0iA4sQ4dUDCAg&uact=5&oq=tazza+calcio&gs_lcp=Cgtwcm9kdWN0cy1jYxADMgcIABAYEIAEMgYIABAWEB4yCAgAEBYQHhAYMggIABAWEB4QGDIICAAQFhAeEBgyCAgAEBYQHhAYMggIABAWEB4QGDIICAAQFhAeEBg6CAgAEIAEELADOgoIABAYEIAEELADOgkIABAeEBgQsAM6CwgAEB4QGBAPELADOgUIABCABDoHCAAQgAQQCjoKCAAQFhAeEBgQD1CVBliiDWCWDmgBcAB4AIABYYgB1gSSAQE3mAEAoAEBwAEByAEK&sclient=products-cc'
link_cover = 'https://www.google.it/search?q=cover+iphone&sa=X&biw=1200&bih=853&tbm=shop&ei=fk-cZLjlLtT1sAfO-5ywDw&ved=0ahUKEwi4nIDgoeb_AhXUOuwKHc49B_YQ4dUDCAg&uact=5&oq=cover+iphone&gs_lcp=Cgtwcm9kdWN0cy1jYxADMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDoKCAAQGBCABBCwAzoJCAAQCBAeELADOgkIABAeEBgQsAM6CwgAEAgQHhAYELADOgcIABCKBRBDUM0FWNEVYNkWaANwAHgAgAFXiAHHB5IBAjE0mAEAoAEBwAEByAEI&sclient=products-cc'
link_lampade = 'https://www.google.it/search?q=lampade+per+scrivania+&hl=it&tbm=shop&sxsrf=AB5stBgJh3vJbyEdaJgzcryPtDf1yK4U1w%3A1688564728077&psb=1&ei=-HOlZMLxA-OMxc8PttOuwA0&ved=0ahUKEwjCza_m2ff_AhVjRvEDHbapC9gQ4dUDCAg&uact=5&oq=lampade+per+scrivania+&gs_lcp=Cgtwcm9kdWN0cy1jYxADMgcIABAYEIAEMgcIABAYEIAEMgcIABAYEIAEMgYIABAWEB4yBggAEBYQHjIICAAQFhAeEBgyCAgAEBYQHhAYMggIABAWEB4QGDIICAAQFhAeEBgyCggAEBYQHhAYEA86CggAEA0QgAQQsAM6DAgAEA0QGBCABBCwAzoLCAAQCBAeEA0QsAM6CwgAEB4QDRAYELADOg0IABAIEB4QDRAYELADOgkIABANEBgQgARQwQdY_g1g2BFoAHAAeACAAW-IAZAGkgEDNC40mAEAoAEBwAEByAEK&sclient=products-cc'
link_zaini = "https://www.google.it/search?tbm=shop&hl=it&psb=1&ved=2ahUKEwiYqcW2nISAAxVCIQYAHSaSD_oQu-kFegQIABAL&q=zaino+eastpak&oq=zaino+eastpak&gs_lcp=Cgtwcm9kdWN0cy1jYxADUABYAGAAaABwAHgAgAEAiAEAkgEAmAEA&sclient=products-cc"
link_xiami = "https://www.google.com/search?q=Xiaomi+Redmi+Note+11S&sa=X&rlz=1C5CHFA_enIT1009IT1009&biw=1132&bih=996&tbm=shop&ei=j1CtZKGNMKuDxc8P29igcA&ved=0ahUKEwjhkJiU2YaAAxWrQfEDHVssCA4Q4dUDCAg&uact=5&oq=Xiaomi+Redmi+Note+11S&gs_lcp=Cgtwcm9kdWN0cy1jYxADMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABFCdBlidBmDQB2gAcAB4AIABygGIAZgCkgEFMS4wLjGYAQCgAQKgAQHAAQE&sclient=products-cc"
link_iphone13 = "https://www.google.com/search?q=iphone+13+128+gb&sa=X&rlz=1C5CHFA_enIT1009IT1009&biw=1792&bih=996&tbm=shop&ei=1FOtZNeqAfGRxc8PqJmNyAI&ved=0ahUKEwjX5Pei3IaAAxXxSPEDHahMAykQ4dUDCAg&uact=5&oq=iphone+13+128+gb&gs_lcp=Cgtwcm9kdWN0cy1jYxADMgcIABCKBRBDMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEMgcIABAYEIAEMgcIABAYEIAEMgcIABAYEIAEOgoIABCKBRCwAxBDOggIABCABBCwA1DQAljsEGCyEmgAcAB4AIABcIgB6wSSAQM3LjGYAQCgAQHAAQHIAQo&sclient=products-cc"

prices = scraping_Google(link_iphone13)
datas = []
datas.append(list_prices_bids(prices,30,0.3))
datas.append(list_prices_bids(prices,30,0.6))
datas.append(list_prices_bids(prices,30,0.9))



In [432]:
datas[0]

[Competitor(id=0, price=779.0, bid=1),
 Competitor(id=1, price=619.0, bid=1),
 Competitor(id=2, price=799.0, bid=1),
 Competitor(id=3, price=749.0, bid=1),
 Competitor(id=4, price=695.0, bid=1),
 Competitor(id=5, price=699.0, bid=1),
 Competitor(id=7, price=689.9, bid=1),
 Competitor(id=9, price=763.37, bid=1),
 Competitor(id=10, price=829.0, bid=1),
 Competitor(id=12, price=604.0, bid=1),
 Competitor(id=16, price=649.99, bid=1),
 Competitor(id=18, price=695.0, bid=1),
 Competitor(id=19, price=636.73, bid=1),
 Competitor(id=20, price=829.0, bid=1),
 Competitor(id=21, price=629.0, bid=1),
 Competitor(id=22, price=683.9, bid=1),
 Competitor(id=25, price=829.0, bid=1),
 Competitor(id=26, price=616.0, bid=1),
 Competitor(id=27, price=695.0, bid=1),
 Competitor(id=28, price=650.0, bid=1),
 Competitor(id=29, price=829.0, bid=1)]

In [429]:
#Blocco di codice dove a mano levo gli outliers.

datas[0].pop(16)

Competitor(id=24, price=799.0, bid=1)

In [433]:
#Per sistemare gli id dopo aver cancellato gli outliers
for i in range(19):
    datas[0][i] = datas[0][i]._replace(id=i)

In [434]:
datas[0] = datas[0][:19]
datas[0]

[Competitor(id=0, price=779.0, bid=1),
 Competitor(id=1, price=619.0, bid=1),
 Competitor(id=2, price=799.0, bid=1),
 Competitor(id=3, price=749.0, bid=1),
 Competitor(id=4, price=695.0, bid=1),
 Competitor(id=5, price=699.0, bid=1),
 Competitor(id=6, price=689.9, bid=1),
 Competitor(id=7, price=763.37, bid=1),
 Competitor(id=8, price=829.0, bid=1),
 Competitor(id=9, price=604.0, bid=1),
 Competitor(id=10, price=649.99, bid=1),
 Competitor(id=11, price=695.0, bid=1),
 Competitor(id=12, price=636.73, bid=1),
 Competitor(id=13, price=829.0, bid=1),
 Competitor(id=14, price=629.0, bid=1),
 Competitor(id=15, price=683.9, bid=1),
 Competitor(id=16, price=829.0, bid=1),
 Competitor(id=17, price=616.0, bid=1),
 Competitor(id=18, price=695.0, bid=1)]

In [442]:
#aggiungo le bid calcolatre tramite i nuovi range, MA con una certa deviazione standard.
#for i in range(len(prices)):
max = max_fun(datas[0])
min = min_fun(datas[0])  
for i in range(19):
    mean_new_bid = bid_generator(max,min,5,1,datas[0][i].price)
    new_bid = np.random.normal(mean_new_bid,0.9)
    if new_bid < 0.5:
        new_bid = 0.5
    if new_bid > 5:
        new_bid = 5
    datas[0][i] = datas[0][i]._replace(bid=round(new_bid,2))


In [443]:
datas[0]

[Competitor(id=0, price=779.0, bid=4.33),
 Competitor(id=1, price=619.0, bid=3.12),
 Competitor(id=2, price=799.0, bid=3.6),
 Competitor(id=3, price=749.0, bid=4.96),
 Competitor(id=4, price=695.0, bid=2.5),
 Competitor(id=5, price=699.0, bid=2.17),
 Competitor(id=6, price=689.9, bid=2.92),
 Competitor(id=7, price=763.37, bid=3.68),
 Competitor(id=8, price=829.0, bid=5),
 Competitor(id=9, price=604.0, bid=1.1),
 Competitor(id=10, price=649.99, bid=1.19),
 Competitor(id=11, price=695.0, bid=2.96),
 Competitor(id=12, price=636.73, bid=1.87),
 Competitor(id=13, price=829.0, bid=5),
 Competitor(id=14, price=629.0, bid=2.05),
 Competitor(id=15, price=683.9, bid=1.63),
 Competitor(id=16, price=829.0, bid=5),
 Competitor(id=17, price=616.0, bid=1.88),
 Competitor(id=18, price=695.0, bid=3.07)]

In [444]:
json_write(datas[0],"ipohne13_0.9.json")


['{"id": 0, "price": 779.0, "bid": 4.33}', '{"id": 1, "price": 619.0, "bid": 3.12}', '{"id": 2, "price": 799.0, "bid": 3.6}', '{"id": 3, "price": 749.0, "bid": 4.96}', '{"id": 4, "price": 695.0, "bid": 2.5}', '{"id": 5, "price": 699.0, "bid": 2.17}', '{"id": 6, "price": 689.9, "bid": 2.92}', '{"id": 7, "price": 763.37, "bid": 3.68}', '{"id": 8, "price": 829.0, "bid": 5}', '{"id": 9, "price": 604.0, "bid": 1.1}', '{"id": 10, "price": 649.99, "bid": 1.19}', '{"id": 11, "price": 695.0, "bid": 2.96}', '{"id": 12, "price": 636.73, "bid": 1.87}', '{"id": 13, "price": 829.0, "bid": 5}', '{"id": 14, "price": 629.0, "bid": 2.05}', '{"id": 15, "price": 683.9, "bid": 1.63}', '{"id": 16, "price": 829.0, "bid": 5}', '{"id": 17, "price": 616.0, "bid": 1.88}', '{"id": 18, "price": 695.0, "bid": 3.07}']
